In [22]:
import time
import datetime
from google.cloud import pubsub
import string
import random
import json
import mysql.connector

In [ ]:
# %%bash 
# pip install -U google.cloud

In [ ]:


def create_connection():
    connection = mysql.connector.connect(user='root', password='root',
                                    host='23.251.142.177',
                                    database='bts')
    return connection

def get_flight_event(index):
  
    query = "SELECT ORIGIN,ORIGIN_AIRPORT_ID, FL_DATE,DEP_DELAY FROM flights LIMIT {};".format(index)
    print(query)
    connection = create_connection()
    cursor = connection.cursor()
    cursor.execute(query)

    data_row = {}
    
    for (ORIGIN,ORIGIN_AIRPORT_ID,FL_DATE,DEP_DELAY) in cursor:
        data_row = {"ORIGIN":ORIGIN,"ORIGIN_AIRPORT_ID":ORIGIN_AIRPORT_ID,"FL_DATE":str(FL_DATE),"DEP_DELAY":DEP_DELAY}
      
    return data_row
 
print(get_flight_event(20))



In [ ]:
TOPIC = "flights"
TIME_FORMAT = '%Y-%m-%d %H:%M:%S'

for i in range(1,10):

    data = get_flight_event(i)

    source_id = str(i)
    source_timestamp = datetime.datetime.now().strftime(TIME_FORMAT)

    client = pubsub.Client()
    topic = client.topic(TOPIC)
    if not topic.exists():
        print ('Creating pub/sub topic {}'.format(TOPIC))
        topic.create()
    else:
        print ('Reusing pub/sub topic {}'.format(TOPIC))

    message = json.dumps(data)
    topic.publish(message=message,source_id = source_id,source_timestamp=source_timestamp)

    print("Message:{} was sent".format(message))

    time.sleep(1)

In [24]:
%%bash

gcloud beta pubsub subscriptions pull --auto-ack default-subscriber

+------------------------------------------------------------------------------------------------------------------------------+-----------------+--------------------------------------------------+
|                                                             DATA                                                             |    MESSAGE_ID   |                    ATTRIBUTES                    |
+------------------------------------------------------------------------------------------------------------------------------+-----------------+--------------------------------------------------+
| eyJPUklHSU4iOiAiTUNPIiwgIkRFUF9ERUxBWSI6IC0xMS4wLCAiRkxfREFURSI6ICIyMDE1LTAxLTAxIiwgIk9SSUdJTl9BSVJQT1JUX0lEIjogIjEzMjA0In0= | 145872218559018 | source_id=1 source_timestamp=2017-08-29 12:30:54 |
+------------------------------------------------------------------------------------------------------------------------------+-----------------+--------------------------------------------------+


In [32]:
client = pubsub.Client()
topic = client.topic(TOPIC)
subscription = topic.subscription("default-subscriber")
message = subscription.pull()
print(message[0][1].data)

{"ORIGIN": "SJU", "DEP_DELAY": -11.0, "FL_DATE": "2015-01-01", "ORIGIN_AIRPORT_ID": "14843"}
